In [1]:
f = open('examples/Profile-20180716T115056', 'r')

In [2]:
import json

profile = json.load(f)

In [3]:
profile_events = []

for row in profile:
    if row['ph'] == 'I' and row['name'] == 'CpuProfile':
        profile_events.append(row)

In [4]:
class Node:
    def __init__(self, name):
        self.name = name
        self.value = 0
        self.children = []
        
    def get_child(self, name):
        for child in self.children:
            if child.name == name:
                return child
        return None
        
    def add(self, stack, value):
        if len(stack) > 0:
            name = stack[0]
            child = self.get_child(name)
            if child is None:
                child = Node(name)
                self.children.append(child)
            child.add(stack[1:], value)    
        else:
            self.value += value
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=2)

In [5]:
def parse_nodes(data):
    nodes = {}
    for node in data['cpuProfile']['nodes']:
        node_id = node['id']
        function_name = node['callFrame']['functionName']
        url = node['callFrame']['url']
        line_number = node['callFrame']['lineNumber']
        children = node.get('children')
        hit_count = node.get('hitCount')
        nodes[node_id] = {'function_name': function_name, 'url': url, 'line_number': line_number, 'hit_count': hit_count, 'children': children}
    return nodes

In [6]:
import copy

root = Node('root')

def callgraph(node_id, nodes, stack):
    node = nodes[node_id] # break in case id doesn't exist
    if node['function_name'] != '(idle)':
        if node['function_name'] == '':
            node['function_name'] = '(anonymous)'
        stack.append(node['function_name'])
        if node['hit_count'] > 0:
            root.add(stack, node['hit_count'])
        if node['children']:
            for child in node['children']:
                callgraph(child, nodes, copy.copy(stack))
    del nodes[node_id]

In [7]:
for profile in profile_events:
    data = profile['args']['data']
    root_id = data['cpuProfile']['nodes'][0]['id']
    nodes = parse_nodes(data)
    callgraph(root_id, nodes, [])

In [8]:
import json

with open('inspector_1.2.json', 'w') as file:
     file.write(root.get_child('(root)').toJSON())